In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
from sklearn import metrics
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import confusion_matrix, accuracy_score

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

In [ ]:
dataset = pd.read_excel("/kaggle/input/hw-data-set/HW_Data_Set.xlsx")
dataset = dataset[dataset['ind_420'] != '?']
dataset = dataset[dataset['ind_422'] != '?']

#dummy

RED = pd.get_dummies(dataset['ind_109'], drop_first =True)
dataset=pd.concat([dataset,RED],axis=1)
dataset.drop(['ind_109'],axis =1,inplace=True)

X = dataset.drop(['20_target','90_target','50_target'], axis=1)
y = dataset.loc[:, dataset.columns == '20_target']

In [ ]:
cate =X.select_dtypes(include='object')
cate

In [ ]:
X['ind_420'].astype(int)

In [ ]:
X['ind_422'].astype(int)

# Catboost

In [ ]:
# 20 target

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.4,random_state=42, shuffle = False)


In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
from catboost import CatBoostRegressor

model = CatBoostRegressor(
    n_estimators = 200,
    loss_function = 'MAE',
    eval_metric = 'RMSE' )

In [ ]:
?model

In [ ]:
model.fit( X_train, y_train, use_best_model=True,  silent=True, plot=True )

In [ ]:
y_pred = model.predict(X_test)
np.sqrt(mean_squared_error(y_test, y_pred))


In [ ]:
# Model Tuning
model_ = CatBoostRegressor()
parameters = {'depth'         : [6,10],
            'learning_rate' : [0.01, 0.05, 0.1],
             'iterations'    : [30, 50, 100]
                 }
grid = GridSearchCV(estimator=model_, param_grid = parameters, cv = 2, n_jobs=-1)
grid.fit(X_train, y_train)

In [ ]:
print("En iyi skor :"+ str(grid.best_score_))
print("En iyi parametresi :"+ str(grid.best_params_))

In [ ]:
cart_tuned = CatBoostRegressor(depth = 6, iterations =30, learning_rate = 0.01)
cart_tuned.fit(X_train,y_train)
y_pred_t = cart_tuned.predict(X_test)
np.sqrt(mean_squared_error(y_test, y_pred_t))